In [6]:
# datetime,open,High,Low,Close,Volume
def pct_delta(a, b):
    return (abs(a - b) / b) * 100


# 定義 FvgStrategy 類別
class FvgStrategy():

    # FVG_DELTA_THRESHOLD 用於定義擺盪區域的閾值
    FVG_DELTA_THRESHOLD = 1

    def __init__(self):
        # 初始化 FVG 策略
        self.fvg_tracker = {
            'delta_p': [],
            'delta_n': [],
        }

    def get_movement_delta(self, chunk, index):
        # 判斷是否為牛市 FVG
        # 找出符合FVG定義K線    
        #                       |               |         |
        #         			--------  close[2]   ---------
        #        		    |       |                | low[3]
        #         			|       |       	      
        #                   |       |
        #                   |       |
        #                   |       |
        #  high[1]   |      |       |
        #        --------    --------   open[2] 
        #        |       |      |
        #        |       | 


        if chunk.high[index] < chunk.low[index + 2] and chunk.high[index] > chunk.open[index + 1] and chunk.low[index + 2] < chunk.close[index + 1]:
            print(pct_delta(chunk.high[index], chunk.low[index + 2]))
            if pct_delta(chunk.high[index], chunk.low[index + 2]) >= self.FVG_DELTA_THRESHOLD: 
                self.fvg_tracker['delta_p'].append({
                    'fvg_high': chunk.low[index + 2],
                    'fvg_low': chunk.high[index],
                    'fvg_timestamp': chunk.index[index],
                    'fvg_chunk_index': index + 2
                })
        # 判斷是否為熊市 FVG
        if chunk.low[index] > chunk.high[index + 2] and chunk.low[index] < chunk.open[index + 1] and chunk.high[index + 2] > chunk.close[index + 1]:
            if pct_delta(chunk.high[index + 2], chunk.low[index]) >= self.FVG_DELTA_THRESHOLD:
                print(chunk)
                self.fvg_tracker['delta_n'].append({
                    'fvg_high': chunk.low[index],
                    'fvg_low': chunk.high[index + 2],
                    'fvg_timestamp': chunk.index[index],
                    'fvg_chunk_index': index + 2
                })

        return self.fvg_tracker
    

![](1.png)

### Load the data

In [7]:
# 導入必要的庫
import pandas as pd  # 用於數據框操作
import pandas_ta as ta  # 用於技術分析指標
import numpy as np  # 用於數學運算
import plotly.graph_objects as go  # 用於繪製圖表
from scipy import stats  # 用於統計分析

# 從CSV文件中讀取比特幣價格數據
filename = "BTCUSDT_15m_2023-4-2023-10_Monthly"
#filename= "BTCUSDT_1h_2023-8-2023-10_Monthly"
pathfile = "C:\Data\Git\Quantitative\BackTrader\Strategy\FVG\\" + filename + ".csv"
df = pd.DataFrame(
    pd.read_csv(pathfile, delimiter=",", index_col="datetime", parse_dates=True)
)


In [8]:
# 找出所有FVG區域

fvg = FvgStrategy()
for i in range(1, len(df)-2):
    fvg.fvg_tracker = fvg.get_movement_delta(df ,i)
    
print(fvg.fvg_tracker)


                        open     high      low    close    volume
datetime                                                         
2023-04-01 08:00:00  28454.8  28491.1  28438.0  28491.1  2728.658
2023-04-01 08:15:00  28491.0  28543.3  28456.9  28479.5  3450.581
2023-04-01 08:30:00  28479.5  28495.7  28401.0  28433.2  2821.510
2023-04-01 08:45:00  28433.1  28444.4  28401.0  28430.1  1972.972
2023-04-01 09:00:00  28430.1  28454.9  28400.0  28423.0  2091.700
...                      ...      ...      ...      ...       ...
2023-10-01 06:45:00  26994.6  27002.7  26985.3  26996.4   447.524
2023-10-01 07:00:00  26996.5  26999.8  26968.1  26975.6   639.795
2023-10-01 07:15:00  26975.5  26986.0  26950.0  26950.0   899.648
2023-10-01 07:30:00  26950.1  26974.6  26942.1  26951.8  1172.090
2023-10-01 07:45:00  26951.9  26964.0  26942.1  26950.9   697.786

[17568 rows x 5 columns]
                        open     high      low    close    volume
datetime                                          

In [9]:
import pandas as pd  # 用於數據框操作
import pandas_ta as ta  # 用於技術分析指標
import numpy as np  # 用於數學運算
import plotly.graph_objects as go  # 用於繪製圖表
from scipy import stats  # 用於統計分析

# 從 df 數據框中選擇特定的行範圍，這裡是從索引300到索引449的範圍，包括這些行
#dfpl = df[300:450]
dfpl = df#[300:450]

# 創建一個新的圖形對象（Figure），其中包括蠟燭圖和樞紐點標記
# fig = go.Figure(data=[
#     go.Candlestick(x=dfpl.index,  # x 軸數據（時間）
#                    open=dfpl['open'],  # 開盤價格
#                    high=dfpl['high'],  # 最高價格
#                    low=dfpl['low'],    # 最低價格
#                    close=dfpl['close']  # 收盤價格
#                   )
# ])

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']),
])
                      
                      
# # 添加散點圖（Scatter plot）用於標記樞紐點的位置
# fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'],  # x 軸為時間，y 軸為樞紐點的位置
#                 mode="markers",  # 使用散點模式
#                 marker=dict(size=5, color="MediumPurple"),  # 散點的大小和顏色
#                 name="pivot"  # 散點圖的名稱
#                 )

# # 添加FVG區域
# #https://plotly.com/python/gantt/#gantt-charts-and-timelines-with-plotlyexpress
# #https://plotly.com/python/waterfall-charts/
for dp in fvg.fvg_tracker['delta_p']:
#     if not dp['fvg_invalidated']:    
    fig.add_shape(
        type="rect",
        x0=dp["fvg_timestamp"],
        x1='2023-10-24 22:00:00',
        y0=dp["fvg_low"],
        y1=dp["fvg_high"],
        fillcolor="green",
        opacity=0.5,
        line=dict(width=0)
    )    
    
# for dp in fvg.fvg_tracker['delta_n']:
# #     if not dp['fvg_invalidated']:    
#     fig.add_shape(
#         type="rect",
#         x0=dp["fvg_timestamp"],
#         x1='2023-09-29 22:00:00',
#         y0=dp["fvg_low"],
#         y1=dp["fvg_high"],
#         fillcolor="red",
#         opacity=0.5,
#         line=dict(width=0)
#     )    


# 更新圖形的布局，隱藏 x 軸上的區域選擇器（xaxis_rangeslider）
fig.update_layout(xaxis_rangeslider_visible=False)

# 顯示圖形
fig.show()


KeyboardInterrupt: 

In [5]:
# #畫出所有FVG區域


# # Charting
# import matplotlib.pyplot as plt
# from mpl_finance import candlestick_ohlc
# import pandas as pd
# import matplotlib.dates as mpl_dates
# import matplotlib.patches as mpatches
# import datetime as dt
# from matplotlib.patches import Rectangle

# def chart_fvg(fvg_data_points, x_current):
#     plt.style.use('ggplot')
#     context = mpl_dates.date2num(x_current)

#     # Extracting Data for plotting
#     data = pd.read_csv('Datasets/Data/ATOM_binance_datetime.csv').head(3500)
#     ohlc = data.loc[:, ['Date', 'Open', 'High', 'Low', 'Close']]
#     ohlc['Date'] = pd.to_datetime(ohlc['Date'])
#     ohlc['Date'] = ohlc['Date'].apply(mpl_dates.date2num)
#     ohlc = ohlc.astype(float)
#     ohlc = ohlc.astype(float)

#     # Creating Subplots
#     fig, ax = plt.subplots()

#     candlestick_ohlc(ax, ohlc.values, width=0.01, colorup='green', colordown='red', alpha=0.8)

#     # Setting labels & titles
#     ax.set_xlabel('Date')
#     ax.set_ylabel('Price')
#     fig.suptitle('btc-binance')

#     # Formatting Date
#     date_format = mpl_dates.DateFormatter('%Y-%m-%d %H:%M:%S')
#     ax.xaxis.set_major_formatter(date_format)
#     fig.autofmt_xdate()

#     fig.tight_layout()

#     for dp in fvg_data_points['delta_p']:
#         if not dp['fvg_invalidated']:
#             start = mpl_dates.date2num(dp['fvg_timestamp'])
#             date_delta = context-start
#             fvg_delta = dp['fvg_high'] - dp['fvg_low']
#             ax.add_patch(Rectangle((start, dp['fvg_low']), date_delta, fvg_delta, color='green', alpha=0.5))

#     for dp in fvg_data_points['delta_n']:
#         if not dp['fvg_invalidated']:
#             start = mpl_dates.date2num(dp['fvg_timestamp'])
#             date_delta = context - start
#             fvg_delta = dp['fvg_high'] - dp['fvg_low']
#             ax.add_patch(Rectangle((start, dp['fvg_low']), date_delta, fvg_delta, color='red', alpha=0.5))


#     plt.show()

